In [ ]:
import re

import numpy as np
import pandas as pd
from Bio import SeqIO
from tqdm import tqdm

In [105]:
def custom_split(line):
    parts = []
    buffer = []
    bracket_level = 0

    for char in line:
        if char == "[":
            bracket_level += 1
        elif char == "]":
            bracket_level -= 1

        if char.isspace() and bracket_level == 0:
            if buffer:
                parts.append("".join(buffer))
                buffer = []
        else:
            buffer.append(char)

    if buffer:
        parts.append("".join(buffer))

    return parts

In [106]:
fasta_file = "data/raw_data/final_tai.fasta"
records = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))

In [107]:
df_final = pd.DataFrame(
    {
        "id": [rec.id for rec in records.values()],
        "sequence": [str(rec.seq) for rec in records.values()],
        "description": [rec.description for rec in records.values()],
    }
)

In [108]:
fasta_file = "data/raw_data/final_tai_top0.1.fasta"
records = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
df_finaltop01 = pd.DataFrame(
    {
        "id": [rec.id for rec in records.values()],
        "sequence": [str(rec.seq) for rec in records.values()],
        "description": [rec.description for rec in records.values()],
    }
)

In [109]:
fasta_file = "data/raw_data/final_tai_top0.1h2_withoutoutliers.fasta"
records = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
df_finaltop01h2 = pd.DataFrame(
    {
        "id": [rec.id for rec in records.values()],
        "sequence": [str(rec.seq) for rec in records.values()],
        "description": [rec.description for rec in records.values()],
    }
)

In [110]:
for index, value in tqdm(df_final["description"].items(), total=len(df_final)):
    value = value.strip()
    name, species, link, tai, *_ = custom_split(value)
    df_final.at[index, "species"] = species
    df_final.at[index, "link"] = link
    df_final.at[index, "tai"] = tai

  0%|          | 0/1493282 [00:00<?, ?it/s]/tmp/ipykernel_4813/1437728193.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[Klebsiella pneumoniae]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.at[index, 'species'] = species
/tmp/ipykernel_4813/1437728193.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'http="https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/026/199/195/GCF_026199195.1_ASM2619919v1"' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.at[index, 'link'] = link
/tmp/ipykernel_4813/1437728193.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[tai=0.277624310878788]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  d

In [ ]:
querypath = "data/raw_data/final_tai.fasta"
refername = "rnpA"
referpath = f"tools/analysis/seqs/{refername}.fasta"

In [215]:
# !makeblastdb -in {querypath} -dbtype nucl

In [216]:
!blastn -query {referpath} -db {querypath} -out tools/analysis/blast/referseq.tsv -outfmt "6"

In [217]:
# XXX
df_blast = pd.read_csv(
    "tools/analysis/blast/referseq.tsv",
    sep="\t",
    header=None,
    names=[
        "query",
        "subject",
        "identity",
        "length",
        "mismatch",
        "gapopen",
        "qstart",
        "qend",
        "sstart",
        "send",
        "evalue",
        "bitscore",
    ],
)

In [203]:
df_ana = df_blast[
    (df_blast["identity"] > 80) & (df_blast["length"] > df_blast["length"].max() * 0.6)
]

In [204]:
df_ana_seq = pd.merge(df_ana, df_final, left_on="subject", right_on="id", how="left")

In [205]:
df_ana_seq["subjectadd"] = "ecoliadd" + df_ana_seq["subject"]

In [206]:
df_ana_seq

,query,subject,identity,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,id,sequence,description,species,link,tai,subjectadd
0,ftsA,lcl|ABCTUH010000003.1_cds_EIE9988303.1_805,100.000,1263,0,0,1,1263,1,1263,0.0,2333,lcl|ABCTUH010000003.1_cds_EIE9988303.1_805,ATGATCAAGGCGACGGACAGAAAACTGGTAGTAGGACTGGAGATTG...,lcl|ABCTUH010000003.1_cds_EIE9988303.1_805 [Es...,[Escherichia coli],"http=""https://ftp.ncbi.nlm.nih.gov/genomes/all...",[tai=0.271973767493961],ecoliaddlcl|ABCTUH010000003.1_cds_EIE9988303.1...
1,ftsA,lcl|NZ_MTPP01000241.1_cds_WP_141030186.1_1055,99.441,895,5,0,1,895,1,895,0.0,1626,lcl|NZ_MTPP01000241.1_cds_WP_141030186.1_1055,ATGATCAAGGCGACGGACAGAAAACTGGTAGTAGGACTGGAGATTG...,lcl|NZ_MTPP01000241.1_cds_WP_141030186.1_1055 ...,[Escherichia coli],"http=""https://ftp.ncbi.nlm.nih.gov/genomes/all...",[tai=0.273887782618595],ecoliaddlcl|NZ_MTPP01000241.1_cds_WP_141030186...
2,ftsA,lcl|BNFW01000001.1_cds_GHL45844.1_694,99.438,889,4,1,1,889,1,888,0.0,1613,lcl|BNFW01000001.1_cds_GHL45844.1_694,ATGATCAAGGCGACGGACAGAAAACTGGTAGTAGGACTGGAGATTG...,lcl|BNFW01000001.1_cds_GHL45844.1_694 [Escheri...,[Escherichia coli],"http=""https://ftp.ncbi.nlm.nih.gov/genomes/all...",[tai=0.314909958867392],ecoliaddlcl|BNFW01000001.1_cds_GHL45844.1_694
3,ftsA,lcl|ABLOSI010000004.1_cds_EKT5066288.1_1698,88.836,1263,139,2,1,1262,1,1262,0.0,1550,lcl|ABLOSI010000004.1_cds_EKT5066288.1_1698,ATGATCAAGGCGACGGACAGAAAACTGGTAGTAGGACTGGAGATTG...,lcl|ABLOSI010000004.1_cds_EKT5066288.1_1698 [S...,[Salmonella enterica],"http=""https://ftp.ncbi.nlm.nih.gov/genomes/all...",[tai=0.33698535412424],ecoliaddlcl|ABLOSI010000004.1_cds_EKT5066288.1...
4,ftsA,lcl|NZ_JALMTA010000004.1_cds_WP_000588466.1_3468,88.440,1263,146,0,1,1263,1,1263,0.0,1524,lcl|NZ_JALMTA010000004.1_cds_WP_000588466.1_3468,ATGATCAAGGCGACGGACAGAAAACTGGTAGTAGGACTGGAGATTG...,lcl|NZ_JALMTA010000004.1_cds_WP_000588466.1_34...,[Escherichia fergusonii],"http=""https://ftp.ncbi.nlm.nih.gov/genomes/all...",[tai=0.279283907978964],ecoliaddlcl|NZ_JALMTA010000004.1_cds_WP_000588...
5,ftsA,lcl|NZ_WHIY01000005.1_cds_WP_003018755.1_4521,88.054,1264,149,2,1,1263,1,1263,0.0,1496,lcl|NZ_WHIY01000005.1_cds_WP_003018755.1_4521,ATGATCAAGGCGACGGACAGAAAACTGGTAGTTGGACTGGAGATTG...,lcl|NZ_WHIY01000005.1_cds_WP_003018755.1_4521 ...,[Citrobacter telavivensis],"http=""https://ftp.ncbi.nlm.nih.gov/genomes/all...",[tai=0.268311319417906],ecoliaddlcl|NZ_WHIY01000005.1_cds_WP_003018755...
6,ftsA,lcl|DAFHFE010000003.1_cds_HBL8875823.1_518,87.094,1263,157,2,1,1263,1,1257,0.0,1424,lcl|DAFHFE010000003.1_cds_HBL8875823.1_518,ATGATCAAGGCGACGGACAGAAAACTGGTAGTTGGACTGGAGATTG...,lcl|DAFHFE010000003.1_cds_HBL8875823.1_518 [En...,[Enterobacter cloacae],"http=""https://ftp.ncbi.nlm.nih.gov/genomes/all...",[tai=0.332896451143815],ecoliaddlcl|DAFHFE010000003.1_cds_HBL8875823.1...
7,ftsA,lcl|NZ_NSCC01000092.1_cds_WP_003018755.1_1387,86.946,1264,163,2,1,1263,1,1263,0.0,1419,lcl|NZ_NSCC01000092.1_cds_WP_003018755.1_1387,ATGATCAAGGCGACGGACAGAAAACTGGTAGTTGGACTGGAGATTG...,lcl|NZ_NSCC01000092.1_cds_WP_003018755.1_1387 ...,[Citrobacter sp. TSA-1],"http=""https://ftp.ncbi.nlm.nih.gov/genomes/all...",[tai=0.243991135233483],ecoliaddlcl|NZ_NSCC01000092.1_cds_WP_003018755...
8,ftsA,lcl|NZ_CP087880.1_cds_WP_015965917.1_2755,86.234,1264,172,2,1,1263,1,1263,0.0,1369,lcl|NZ_CP087880.1_cds_WP_015965917.1_2755,ATGATCAAGGCGACGGACAGAAAACTGGTAGTTGGACTGGAGATTG...,lcl|NZ_CP087880.1_cds_WP_015965917.1_2755 [Pse...,[Pseudocitrobacter corydidari],"http=""https://ftp.ncbi.nlm.nih.gov/genomes/all...",[tai=0.326085203818039],ecoliaddlcl|NZ_CP087880.1_cds_WP_015965917.1_2755
9,ftsA,lcl|NZ_WMJZ01000002.1_cds_WP_155106801.1_1624,85.986,1263,177,0,1,1263,1,1263,0.0,1352,lcl|NZ_WMJZ01000002.1_cds_WP_155106801.1_1624,ATGATCAAGGCGACGGACAGAAAACTGGTAGTTGGACTGGAGATTG...,lcl|NZ_WMJZ01000002.1_cds_WP_155106801.1_1624 ...,[Intestinirhabdus alba],"http=""https://ftp.ncbi.nlm.nih.gov/genomes/all...",[tai=0.306473749982561],ecoliaddlcl|NZ_WMJZ01000002.1_cds_WP_155106801...


In [207]:
len(df_ana_seq)

47

In [208]:
df_finaltop01[df_finaltop01["id"].isin(df_ana_seq["subject"].unique())]

,id,sequence,description


In [209]:
df_finaltop01[df_finaltop01["id"].isin(df_ana_seq["subjectadd"].unique())]

,id,sequence,description


In [210]:
df_finaltop01h2[df_finaltop01h2["id"].isin(df_ana_seq["subject"].unique())]

,id,sequence,description


In [211]:
df_finaltop01h2[df_finaltop01h2["id"].isin(df_ana_seq["subjectadd"].unique())]

,id,sequence,description


In [212]:
with open(f"tools/analysis/seqs/{refername}_similar.fasta", "w") as f:
    count = []

    for index, row in df_ana_seq.iterrows():
        f.write(f">{row['id']}\n{row['sequence']}\n")
        count.append(row["sequence"])

In [213]:
from collections import Counter

print(Counter(count))

Counter({'ATGATCAAGGCGACGGACAGAAAACTGGTAGTAGGACTGGAGATTGGTACCGCGAAGGTTGCCGCTTTAGTAGGGGAAGTTCTGCCCGACGGTATGGTCAATATCATTGGCGTGGGCAGCTGCCCGTCGCGTGGTATGGATAAAGGCGGGGTGAACGACCTCGAATCCGTGGTCAAGTGCGTACAACGCGCCATTGACCAGGCAGAATTGATGGCAGATTGTCAGATCTCTTCGGTATATCTGGCGCTTTCTGGTAAGCACATCAGCTGCCAGAATGAAATTGGTATGGTGCCTATTTCTGAAGAAGAAGTGACGCAAGAAGATGTGGAAAACGTCGTCCATACCGCGAAATCGGTGCGTGTGCGCGATGAGCATCGTGTGCTGCATGTGATCCCGCAAGAGTATGCGATTGACTATCAGGAAGGGATCAAGAATCCGGTAGGACTTTCGGGCGTGCGGATGCAGGCAAAAGTGCACCTGATCACATGTCACAACGATATGGCGAAAAACATCGTCAAAGCGGTTGAACGTTGTGGGCTGAAAGTTGACCAACTGATATTTGCCGGACTGGCATCAAGTTATTCGGTATTGACGGAAGATGAACGTGAACTGGGTGTCTGCGTCGTCGATATCGGTGGTGGTACAATGGATATCGCCGTTTATACCGGTGGGGCATTGCGCCACACTAAGGTAATTCCTTATGCTGGCAATGTCGTGACCAGTGATATCGCTTACGCCTTTGGCACGCCGCCAAGCGACGCCGAAGCGATTAAAGTTCGCCACGGTTGTGCGCTGGGTTCCATCGTTGGAAAAGATGAGAGCGTGGAAGTGCCGAGCGTAGGTGGTCGTCCGCCACGGAGTCTGCAACGTCAGACACTGGCAGAGGTGATCGAGCCGCGCTATACCGAGCTGCTCAACCTGGTCAACGAAGAGATATTGCAGTTGCAGGAAAAGCTTCGCCAACAAGGGGTTAAACATCACCTGGCGGCA